In [1]:
#%% New run

import pandas as pd
import numpy as np

# Get data
cars = pd.read_csv(r'/Users/tiepnguyen/cars depreciation from 2015 to 2019 cleaned including body_type.csv')

#cars = cars[cars['SUV / Crossover'] == 1]
#cars = cars[cars['Sedan'] == 1]
#cars = cars[cars['Pickup Truck'] == 1]

#cars = cars[cars['mass_market'] == 1]

cars = cars.drop(['Unnamed: 0','year','Pickup Truck','SUV / Crossover','Sedan','mass_market','other','niche'], axis=1)

# load the dataset
X = cars.drop('depreciation_percent_from_2020',axis=1)
y = cars.depreciation_percent_from_2020

# Split X and y into training and test set in 75:25 ratio
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25 , random_state=1)

In [3]:
#%% Tuned Random Forest Regression
from numpy import mean
from numpy import std
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
from sklearn.pipeline import make_pipeline

# Instantiate model with 10 decision trees
RF_tuned_regressor = RandomForestRegressor(n_estimators = 200, min_weight_fraction_leaf=0.0, min_samples_split=30, min_samples_leaf=2, max_features="auto", max_depth=None, bootstrap=(True), random_state = 42)

# Train the model on training data
RF_tuned_regressor.fit(X_train, y_train)

RF_tuned_test_RMSE = mean_squared_error(y_test, RF_tuned_regressor.predict(X_test), squared=False)
RF_tuned_train_RMSE = mean_squared_error(y_train, RF_tuned_regressor.predict(X_train), squared=False)

RF_tuned_test_MAE = mean_absolute_error(y_test, RF_tuned_regressor.predict(X_test))
RF_tuned_train_MAE = mean_absolute_error(y_train, RF_tuned_regressor.predict(X_train))

"""
RF_tuned_test_MAPE = mean_absolute_percentage_error(y_test, RF_tuned_regressor.predict(X_test))
RF_tuned_train_MAPE = mean_absolute_percentage_error(y_train, RF_tuned_regressor.predict(X_train))
"""

# Adjusted R-squared
RF_tuned_test_Adjusted_R2 = 1 - (1-r2_score(y_test, RF_tuned_regressor.predict(X_test))) * ( len(y_test) - 1 ) / ( len(y_test) - X_test.shape[1] - 1 )
RF_tuned_train_Adjusted_R2 = 1 - (1-r2_score(y_train, RF_tuned_regressor.predict(X_train))) * ( len(y_train) - 1 ) / ( len(y_train) - X_train.shape[1] - 1 )

print('Train MAE: ', RF_tuned_train_MAE)
#print('Train MAPE: ', RF_tuned_train_MAPE)
print('Train RMSE: ', RF_tuned_train_RMSE)
print('Train Adjusted R2: ', RF_tuned_train_Adjusted_R2)

print('Test MAE: ', RF_tuned_test_MAE)
#print('Test MAPE: ', RF_tuned_test_MAPE)
print('Test RMSE: ', RF_tuned_test_RMSE)
print('Test Adjusted R2: ', RF_tuned_test_Adjusted_R2)


Train MAE:  5.890970787933676
Train RMSE:  7.587989792077271
Train Adjusted R2:  0.6315677172835596
Test MAE:  6.05445871811911
Test RMSE:  7.8050672346388525
Test Adjusted R2:  0.6102909258084999


In [ ]:
from sklearn.inspection import PartialDependenceDisplay
from time import time

common_params = {
    "subsample": 50,
    "n_jobs": 2,
    "centered": True,
    "grid_resolution": 40,
    "random_state": 0,
}

print("Computing partial dependence plots...")
tic = time()
display = PartialDependenceDisplay.from_estimator(
    RF_tuned_regressor,
    X_train,
    features=["highway_fuel_economy", "city_fuel_economy", "height", "length", "maximum_seating"],
    kind="average",
    **common_params,
)

print(f"done in {time() - tic:.3f}s")
display.figure_.suptitle(
    "Partial dependence of used cars' depreciation rate on 5 most important features\n"
    "for the Carguru used cars dataset, with Random Forest Regression"
)
display.figure_.subplots_adjust(wspace=0.4, hspace=0.4)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.inspection import PartialDependenceDisplay
from time import time

common_params = {
    "subsample": 50,
    "n_jobs": 2,
    "centered": True,
    "grid_resolution": 40,
    "random_state": 0,
}

print("Computing partial dependence plots...")
tic = time()
_, ax = plt.subplots(ncols=3, figsize=(9, 4))

# Note that we could have called the method `from_estimator` three times and
# provide one feature, one kind of plot, and one axis for each call.
display = PartialDependenceDisplay.from_estimator(
    RF_tuned_regressor,
    X_train,
    features=["highway_fuel_economy", "city_fuel_economy", ("highway_fuel_economy", "city_fuel_economy")],
    kind=["both", "both", "average"],
    ax=ax,
    **common_params,
)

print(f"done in {time() - tic:.3f}s")
display.figure_.suptitle(
    "Partial dependence of house value on non-location features\n"
    "for the California housing dataset, with Gradient Boosting"
)
display.figure_.subplots_adjust(wspace=0.4, hspace=0.3)